In [1]:
import fiona
import geopandas as gpd
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import matplotlib.patches as mpatches
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling,transform
from rasterio.crs import CRS
from rasterio import plot
import xarray as xr
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable


def reproject_raster(raster,dst_crs,band=1):
    transform, width, height = calculate_default_transform(raster.crs, dst_crs, raster.width, raster.height, *raster.bounds)
    kwargs = raster.meta.copy()

    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    out_np = raster.read(band)
    out_np = np.ones((height,width))*kwargs['nodata']
    reproject(
    source=rio.band(raster, band),
    destination=out_np,
    src_transform=raster.transform,
    src_crs=raster.crs,
    dst_transform=transform,
    dst_crs=dst_crs,
    resampling=Resampling.nearest)
    return out_np,kwargs

ImportError: DLL load failed while importing ogrext: The specified module could not be found.

In [2]:
ecoregion_gdf = gpd.read_file('data/terr_eco_regions/tnc_terr_ecoregions.shp')

In [3]:
aggregated_biome_data = pd.read_csv('data/aggregated_biomes_data_20210613.csv')

In [4]:
ecoregion_merged_gdf = ecoregion_gdf.merge(aggregated_biome_data[['biome','aggregated biome 1']],left_on='WWF_MHTNAM',right_on='biome')

In [5]:
aggregated_biomes_gpd = ecoregion_merged_gdf.dissolve(by='aggregated biome 1')
aggregated_biomes_gpd = aggregated_biomes_gpd.loc[aggregated_biomes_gpd.index[[0,1,3,4,5,6,7,8,2]]]

In [6]:
crs = ccrs.Mollweide()
crs_proj4 = crs.proj4_init
aggregated_biomes_gdf_ae = aggregated_biomes_gpd.to_crs(crs_proj4)
aggregated_biomes_gdf_ae['color'] = ['#065535','#F2D16B','#82490b','#5aa15f','#b8f500','#5ece1d','#fc9003','#a866e9','#dddddd']
aggregated_biomes_gdf_ae.index = aggregated_biomes_gdf_ae.index.str.replace(',',',\n')

In [24]:

fig = plt.figure(dpi=300,figsize=(12,6))
ax = plt.subplot(2,1,1,projection=crs)
q = cm.get_cmap('tab10')
geos = []
ax.coastlines()
ax.set_title('Biomes')
biome_order = [7,0,3,5,4,6,2,1,8]
for i,row in aggregated_biomes_gdf_ae.iloc[biome_order].iterrows():
    ax.add_geometries(row['geometry'],crs=crs,facecolor=row['color'])
    geos.append(mpatches.Rectangle((0, 0), 1, 1, facecolor=row['color']))
plt.legend(geos,aggregated_biomes_gdf_ae.index[biome_order],ncol=2,bbox_to_anchor=(-0.012,1))


crop_xr = xr.open_rasterio('data/cropland.tif')
ax = plt.subplot(2,3,4,projection=crs)
crop_xr = crop_xr.mean(dim='band')
i = ax.imshow(crop_xr.where(crop_xr>=0),cmap='Greens',interpolation='none',transform=ccrs.PlateCarree())
ax.coastlines()
ax.set_title('Croplands')
b = plt.colorbar(i,ax=ax,shrink=0.6, aspect=20*0.6,ticks=np.arange(0,1.2,0.2))
b.formatter = ticker.PercentFormatter(1,0)
b.update_ticks()

pasture_xr = xr.open_rasterio('data/pasture.tif')
ax = plt.subplot(2,3,5,projection=crs)
pasture_xr = pasture_xr.mean(dim='band')
i = ax.imshow(pasture_xr.where(pasture_xr>=0),cmap='Reds',interpolation='none',transform=ccrs.PlateCarree())
# i = ax.contourf(pasture_xr.where(pasture_xr>=0),cmap='Reds',transform=ccrs.PlateCarree())
ax.coastlines()
ax.set_title('Pasture')
b = plt.colorbar(i,ax=ax,shrink=0.6, aspect=20*0.6,ticks=np.arange(0,1.2,0.2))
b.formatter = ticker.PercentFormatter(1,0)
b.update_ticks()

ax = plt.subplot(2,3,6,projection=crs)
i = ax.imshow(1- pasture_xr.where(pasture_xr>=0)-crop_xr.where(crop_xr>=0),cmap='Blues',interpolation='none',transform=ccrs.PlateCarree())
# i = ax.contourf(pasture_xr.where(pasture_xr>=0),cmap='Reds',transform=ccrs.PlateCarree())
ax.coastlines()
ax.set_title('Other (wilderness,\nmanaged forests, etc.)')
b = plt.colorbar(i,ax=ax,shrink=0.6, aspect=20*0.6,ticks=np.arange(0,1.2,0.2))
b.formatter = ticker.PercentFormatter(1,0)
b.update_ticks()

plt.savefig('results/biome_maps.svg',dpi=300)